# Graphein Protein Structure Dataloaders
## PyTorch Geometric Datasets

[API Reference](https://graphein.ai/modules/graphein.ml.html#graphein.ml.datasets.torch_geometric_dataset)

Graphein provides three dataset classes for working with [PyTorch Geometric](https://pytorch-geometric.readthedocs.io/en/latest/):

* [`ProteinGraphDataset`]() - For processing large datasets that can't be kept in memory
* [`InMemoryProteinGraphDataset`]() - For smaller datasets that can be kept in memory
* [`ProteinGraphListDataset`]() - For creating a dataset from a list of pre-computed PyTorch Geometric graphs.

Both `ProteinGraphDataset` and `InMemoryGraphDataset` will take care of downloading structures from either the [RCSB PDB](https://www.rcsb.org/), [EBI AlphaFold database](https://alphafold.com/), or both!
`ProteinGraphListDataset` is a lightweight alternative for creating a dataset from a collection of graphs you have pre-computed.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/a-r-j/graphein/blob/master/notebooks/dataloader_tutorial.ipynb)

In [1]:
# Install graphein if necessary
# !pip install graphein

# Install torch if necessary. See https://pytorch.org/get-started/locally/
# pip install torch==1.11.0

# Install torch geometric if necessary. See: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
# pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.11.0+cpu.html

### ProteinGraphDataset

[API Reference](https://graphein.ai/modules/graphein.ml.html#graphein.ml.datasets.torch_geometric_dataset.ProteinGraphDataset)

`ProteinGraphDataset` will download structures from the PDB/AlphafoldDB, process the structures into graphs according to a `ProteinGraphConfig`.

#### Parameters

```python
ProteinGraphDataset(
        root: str,                                                             
        # Root directory where the dataset should be saved.
        name: str,                                                             
        # Name of the dataset. Will be saved to ``data_$name.pt``.
        pdb_paths:Optional[List[str]] =None,
        # List of full path of pdb files to load.
        pdb_codes: Optional[List[str]] = None,                                 
        #  List of PDB codes to download and parse from the PDB.
        uniprot_ids: Optional[List[str]] = None,                               
        # List of Uniprot IDs to download and parse from Alphafold Database
        graph_label_map: Optional[Dict[str, torch.Tensor]] = None,             
        # Dictionary mapping PDB/Uniprot IDs to graph-level labels.
        node_label_map: Optional[Dict[str, torch.Tensor]] = None,              
        # Dictionary mapping PDB/Uniprot IDs to node-level labels.
        chain_selection_map: Optional[Dict[str, List[str]]] = None,            
        # Dictionary mapping PDB/Uniprot IDs to the desired chains in the PDB files
        graphein_config: ProteinGraphConfig = ProteinGraphConfig(),            
        # Protein graph construction config
        graph_format_convertor: GraphFormatConvertor = GraphFormatConvertor(   
            src_format="nx", dst_format="pyg"
        ),
        # Conversion handler for graphs
        graph_transformation_funcs: Optional[List[Callable]] = None,           
        # List of functions that consume a nx.Graph and return a nx.Graph. Applied to graphs after construction but before conversion to pyg
        transform: Optional[Callable] = None,                                  
        # A function/transform that takes in a torch_geometric.data.Data object and returns a transformed version. The data object will be transformed before every access.
        pdb_transform: Optional[List[Callable]] = None,
        pre_transform: Optional[Callable] = None,                              
        # A function/transform that takes in a torch_geometric.data.Data object and returns a transformed version. The data object will be transformed before being saved to disk
        pre_filter: Optional[Callable] = None,                                 
        # A function that takes in a torch_geometric.data.Data object and returns a boolean value, indicating whether the data object should be included in the final dataset
        num_cores: int = 16,                                                   
        # Number of cores to use for multiprocessing of graph construction
        af_version: int = 2,                                                   
        #  Version of AlphaFoldDB structures to use,
    )
```


#### Directory Structure
Creating a ``ProteinGraphDataset`` will create two directories under ``root``:

* ``root/raw`` - Contains raw PDB files which are downloaded
* ``root/processed`` - Contains processed graphs (in ``pytorch_geometric.data.Data`` format) saved as ``$PDB.pt / $UNIPROT_ID.pt``

In [2]:
import torch
from graphein.ml import ProteinGraphDataset
import graphein.protein as gp

# Create some labels
g_labels = [torch.randn([5])] * 5
n_labels = [torch.randn([10])] * 5

# Select some chains
chain_selections = ["all", "A", "all", "all", "all"]


# Create the dataset
ds = ProteinGraphDataset(
    root = "../graphein/ml/datasets/test",
    pdb_codes=["3eiy", "4hhb", "1lds"],
    uniprot_ids=["Q5VSL9", "Q8W3K0"],
    graph_labels=g_labels,
    node_labels=n_labels,
    chain_selections=chain_selections,
    graphein_config=gp.ProteinGraphConfig()
)

[01/30/23 14:22:09] WARNING  To use the Graphein submodule                ]8;id=75456;file:///home/atj39/github/graphein/graphein/protein/visualisation.py\visualisation.py]8;;\:]8;id=973046;file:///home/atj39/github/graphein/graphein/protein/visualisation.py#35\35]8;;\
                             graphein.protein.visualisation, you need to                     
                             install: pytorch3d                                              
                             To do so, use the following command: conda                      
                             install -c pytorch3d pytorch3d                                  

                    WARNING  To use the Graphein submodule                       ]8;id=992158;file:///home/atj39/github/graphein/graphein/protein/meshes.py\meshes.py]8;;\:]8;id=729074;file:///home/atj39/github/graphein/graphein/protein/meshes.py#29\29]8;;\
                             graphein.protein.meshes, you need to install:                   
                             pytorch3d                                                       
                             To do so, use the following command: conda install              
                             -c pytorch3d pytorch3d                                          

Processing...
100%|██████████| 1/1 [00:00<00:00,  1.26it/s]
Done!


In [3]:
ds.node_label_map

{0: tensor([-1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642, -0.6204,  1.1605,
          0.3182, -0.1100]),
 1: tensor([-1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642, -0.6204,  1.1605,
          0.3182, -0.1100]),
 2: tensor([-1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642, -0.6204,  1.1605,
          0.3182, -0.1100]),
 3: tensor([-1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642, -0.6204,  1.1605,
          0.3182, -0.1100]),
 4: tensor([-1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642, -0.6204,  1.1605,
          0.3182, -0.1100])}

In [4]:
# Create a dataloader from dataset and inspect a batch
from torch_geometric.loader import DataLoader

dl = DataLoader(ds, batch_size=2, shuffle=True, drop_last=True)
for i in dl:
    print(i)
    print("Graph labels: ", i.graph_y)
    print("Node labels: ", i.node_y)
    break

DataBatch(edge_index=[2, 1233], node_id=[2], coords=[2], name=[2], dist_mat=[2], num_nodes=1235, graph_y=[10], node_y=[20], batch=[1235], ptr=[3])
Graph labels:  tensor([ 0.3875,  0.1054, -0.4155,  0.6837, -0.2177,  0.3875,  0.1054, -0.4155,
         0.6837, -0.2177])
Node labels:  tensor([-1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642, -0.6204,  1.1605,
         0.3182, -0.1100, -1.0872,  0.0402,  0.2413, -1.7507,  0.7504, -0.4642,
        -0.6204,  1.1605,  0.3182, -0.1100])


#### Load from local path


Creating a ``ProteinGraphDataset`` from a list of full path of pdb files:

* ``root/raw`` - Will be empty since no pdb files are downloaded
* ``root/processed`` - Contains processed graphs (in ``pytorch_geometric.data.Data`` format) saved as ``$PDB.pt / $UNIPROT_ID.pt``

In [5]:
# import sys
# sys.path.append('../')  # add system path for python

import os 
from graphein.protein.config import ProteinGraphConfig
from graphein.ml import ProteinGraphDataset, ProteinGraphListDataset
import torch 

local_dir = "../tests/protein/test_data/"
pdb_paths = [os.path.join(local_dir, pdb_path) for pdb_path in os.listdir(local_dir) if pdb_path.endswith(".pdb")]
print(pdb_paths)

# let's load local dataset from local_dir!
ds = ProteinGraphDataset(
    root = "../graphein/ml/datasets/test",
    pdb_paths = pdb_paths,
    graphein_config=ProteinGraphConfig(),
)

Processing...


['../tests/protein/test_data/4hhb.pdb', '../tests/protein/test_data/1lds.pdb', '../tests/protein/test_data/alphafold_structure.pdb']


100%|██████████| 1/1 [00:00<00:00,  3.09it/s]
Done!


In [6]:
# Create a dataloader from dataset and inspect a batch
from torch_geometric.loader import DataLoader
dl = DataLoader(ds, batch_size=2, shuffle=True, drop_last=True)
for i in dl:
    print(i)
    break

DataBatch(edge_index=[2, 951], node_id=[2], coords=[2], name=[2], dist_mat=[2], num_nodes=956, batch=[956], ptr=[3])


### InMemoryProteinGraphDataset

[API Reference](https://graphein.ai/modules/graphein.ml.html#graphein.ml.datasets.torch_geometric_dataset.InMemoryProteinGraphDataset)

#### Parameters
```python
InMemoryProteinGraphDataset(
        root: str,                                                             
        # Root directory where the dataset should be saved.
        name: str,                                                             
        # Name of the dataset. Will be saved to ``data_$name.pt``.
        pdb_paths:Optional[List[str]] =None,
        # List of full path of pdb files to load.
        pdb_codes: Optional[List[str]] = None,                                 
        #  List of PDB codes to download and parse from the PDB.
        uniprot_ids: Optional[List[str]] = None,                               
        # List of Uniprot IDs to download and parse from Alphafold Database
        graph_label_map: Optional[Dict[str, torch.Tensor]] = None,             
        # Dictionary mapping PDB/Uniprot IDs to graph-level labels.
        node_label_map: Optional[Dict[str, torch.Tensor]] = None,              
        # Dictionary mapping PDB/Uniprot IDs to node-level labels.
        chain_selection_map: Optional[Dict[str, List[str]]] = None,            
        # Dictionary mapping PDB/Uniprot IDs to the desired chains in the PDB files
        graphein_config: ProteinGraphConfig = ProteinGraphConfig(),            
        # Protein graph construction config
        graph_format_convertor: GraphFormatConvertor = GraphFormatConvertor(   
            src_format="nx", dst_format="pyg"
        ),
        # Conversion handler for graphs
        graph_transformation_funcs: Optional[List[Callable]] = None,           
        # List of functions that consume a nx.Graph and return a nx.Graph. Applied to graphs after construction but before conversion to pyg
        transform: Optional[Callable] = None,                                  
        # A function/transform that takes in a torch_geometric.data.Data object and returns a transformed version. The data object will be transformed before every access.
        pdb_transform: Optional[List[Callable]] = None,
        pre_transform: Optional[Callable] = None,                              
        # A function/transform that takes in a torch_geometric.data.Data object and returns a transformed version. The data object will be transformed before being saved to disk
        pre_filter: Optional[Callable] = None,                                 
        # A function that takes in a torch_geometric.data.Data object and returns a boolean value, indicating whether the data object should be included in the final dataset
        num_cores: int = 16,                                                   
        # Number of cores to use for multiprocessing of graph construction
        af_version: int = 2,                                                   
        #  Version of AlphaFoldDB structures to use,
    )
```

#### Directory Structure
Creating an ``InMemoryProteinGraphDataset`` will create two directories under ``root``:
* ``root/raw`` - Contains raw PDB files
* ``root/processed`` - Contains processed datasets saved as ``data_{name}.pt``

In [7]:
from graphein.ml import InMemoryProteinGraphDataset

g_lab_map = {"3eiy": 1, "4hhb": 2, "Q5VSL9": 3, "1lds": 10, "2ll6": 4}
node_lab_map = {"3eiy": 1, "4hhb": 2, "Q5VSL9": 3, "1lds": 10, "2ll6": 4}
chain_selection_map = {"4hhb": "A"}

ds = InMemoryProteinGraphDataset(
    root = "../graphein/ml/datasets/test",
    name="test",
    pdb_codes=["3eiy", "4hhb", "1lds", "2ll6"],
    uniprot_ids=["Q5VSL9"],
    graph_label_map=g_lab_map,
    node_label_map=node_lab_map,
    chain_selection_map=chain_selection_map
)

Constructing Graphs...


Processing...
100%|██████████| 5/5 [00:00<00:00,  5.05it/s]

Converting Graphs...
Assigning graph Labels...
Assigning node Labels...
Saving Data...
Done!



Done!


In [8]:
# Create a dataloader from dataset and inspect a batch
dl = DataLoader(ds, batch_size=2, shuffle=True, drop_last=True)
for i in dl:
    print(i)
    break

DataBatch(edge_index=[2, 1009], node_id=[2], coords=[2], name=[2], dist_mat=[2], graph_y=[2], node_y=[2], num_nodes=1011, batch=[1011], ptr=[3])


#### Load from local path


Creating an ``InMemoryProteinGraphDataset`` from a list of full path of pdb files:

* ``root/raw`` - Will be empty since no pdb files are downloaded
* ``root/processed`` - Contains processed datasets saved as ``data_{name}.pt``


In [9]:
from graphein.ml.datasets.torch_geometric_dataset import InMemoryProteinGraphDataset


local_dir = "../tests/protein/test_data/"
pdb_paths = [os.path.join(local_dir, pdb_path) for pdb_path in os.listdir(local_dir) if pdb_path.endswith(".pdb")]
print(pdb_paths)

# let's load local dataset from local_dir!
ds = InMemoryProteinGraphDataset(
    root = "../graphein/ml/datasets/test",
    name = "test",
    pdb_paths = pdb_paths,
)


['../tests/protein/test_data/4hhb.pdb', '../tests/protein/test_data/1lds.pdb', '../tests/protein/test_data/alphafold_structure.pdb']


In [10]:
# Create a dataloader from dataset and inspect a batch
dl = DataLoader(ds, batch_size=2, shuffle=True, drop_last=True)
for i in dl:
    print(i)
    break

DataBatch(edge_index=[2, 236], node_id=[2], coords=[2], name=[2], dist_mat=[2], graph_y=[2], node_y=[2], num_nodes=238, batch=[238], ptr=[3])


### ProteinGraphListDataset

[API Reference](https://graphein.ai/modules/graphein.ml.html#graphein.ml.datasets.torch_geometric_dataset.ProteinGraphListDataset)

The `ProteinGraphListDataset` class is a lightweight class for wrapping a list of pre-computed `pytorch_geometric.data.Data` graphs.

#### Parameters

```python
ProteinGraphListDataset(
    root: str,                              # Root directory where the dataset is stored.
    data_list: List[Data],                  # List of protein graphs as PyTorch Geometric Data objects.
    name: str,                              # Name of dataset. Data will be saved as ``data_{name}.pt``.
    transform: Optional[Callable]=None      # A function/transform that takes in a torch_geometric.data.Data object and returns a transformed version. The data object will be transformed before every access.
    )
```

In [11]:
from graphein.ml import ProteinGraphListDataset, GraphFormatConvertor
import graphein.protein as gp

# Construct graphs
graphs = gp.construct_graphs_mp(
    pdb_code_it=["3eiy", "4hhb", "1lds", "2ll6"],
    return_dict=False
    )

# do some transformation
graphs = [gp.extract_subgraph_from_chains(g, ["A"]) for g in graphs]

# Convert to PyG Data format
convertor = GraphFormatConvertor(src_format="nx", dst_format="pyg")
graphs = [convertor(g) for g in graphs]

# Create dataset
ds = ProteinGraphListDataset(root=".", data_list=graphs, name="list_test")

100%|██████████| 4/4 [00:03<00:00,  1.23it/s]


[01/30/23 14:22:40] DEBUG    Found 174 nodes in the chain subgraph.          ]8;id=761748;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=711551;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#266\266]8;;\

                    DEBUG    Creating subgraph from nodes: ['A:PHE:16',       ]8;id=911253;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=919579;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#57\57]8;;\
                             'A:TYR:52', 'A:LYS:35', 'A:GLN:141', 'A:ALA:33',                
                             'A:TYR:130', 'A:PRO:28', 'A:GLY:83', 'A:ILE:20',                
                             'A:THR:118', 'A:ALA:88', 'A:PRO:128',                           
                             'A:ASP:15', 'A:ILE:124', 'A:PRO:69', 'A:GLY:92',                
                             'A:VAL:6', 'A:PRO:53', 'A:LYS:149', 'A:GLN:61',                 
                             'A:ASP:168', 'A:ALA:82', 'A:GLY:38', 'A:THR:97',                
                             'A:GLU:27', 'A:THR:167', 'A:GLY:148',                           
                             'A:LEU:63', 'A:ASP:112', 'A:ILE:159',                           
                             'A:LEU:81', 'A:VAL:42', 'A:ASP:66', 'A:VAL:114',                
                             'A:PRO:79', 'A:THR:48', 'A:ASP:11', 'A:VAL:170',                
                             'A:ALA:171', 'A:LYS:143', 'A:ASP:126',                          
                             'A:PHE:173', 'A:VAL:70', 'A:ILE:22',                            
                             'A:PHE:138', 'A:LYS:105', 'A:LYS:175',                          
                             'A:PRO:23', 'A:LYS:95', 'A:ALA:161',                            
                             'A:HIS:163', 'A:PRO:7', 'A:GLU:21', 'A:HIS:111',                
                             'A:GLN:134', 'A:ASP:157', 'A:LYS:164',                          
                             'A:ILE:19', 'A:ILE:46', 'A:PRO:110',                            
                             'A:LYS:132', 'A:VAL:54', 'A:LYS:147',                           
                             'A:ALA:24', 'A:CYS:115', 'A:ASP:43',                            
                             'A:LYS:113', 'A:HIS:137', 'A:LYS:174',                          
                             'A:PHE:58', 'A:ILE:75', 'A:SER:26', 'A:GLY:9',                  
                             'A:LEU:73', 'A:GLY:47', 'A:LEU:80', 'A:GLU:154',                
                             'A:PRO:60', 'A:SER:2', 'A:PHE:45', 'A:ASN:17',                  
                             'A:ALA:144', 'A:THR:76', 'A:VAL:86', 'A:VAL:18',                
                             'A:PRO:116', 'A:ASP:68', 'A:ARG:89', 'A:ASN:55',                
                             'A:VAL:74', 'A:GLU:140', 'A:SER:4', 'A:VAL:85',                 
                             'A:GLU:165', 'A:GLY:169', 'A:TRP:156',                          
                             'A:ASN:5', 'A:SER:64', 'A:VAL:107', 'A:SER:123',                
                             'A:PRO:13', 'A:PHE:3', 'A:VAL:72', 'A:THR:62',                  
                             'A:MET:50', 'A:TRP:150', 'A:VAL:153',                           
                             'A:VAL:102', 'A:ILE:135', 'A:LEU:39',                           
                             'A:ALA:129', 'A:LEU:37', 'A:VAL:41',                            
                             'A:LEU:121', 'A:TYR:142', 'A:TYR:56',                           
                             'A:GLU:146', 'A:ASP:98', 'A:ALA:108',                           
                             'A:GLY:158', 'A:ALA:8', 'A:SER:84', 'A:GLN:14',                 
                             'A:ARG:87', 'A:MET:117', 'A:TYR:31', 'A:GLU:32',                
                             'A:LEU:145', 'A:GLU:99', 'A:ARG:51',                            
                             'A:ALA:162', 'A:LYS:136', 'A:ALA:36',                           
                             'A:GLY:67', 'A:LEU:94', 'A:ALA:90', 'A:LEU:131',                
                             'A:GLN:25', 'A:GLY:101', 'A:SER:10

                    DEBUG    Found 141 nodes in the chain subgraph.          ]8;id=386632;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=583449;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#266\266]8;;\

                    DEBUG    Creating subgraph from nodes: ['A:HIS:87',       ]8;id=547065;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=574779;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#57\57]8;;\
                             'A:GLY:59', 'A:VAL:96', 'A:THR:118', 'A:ALA:88',                
                             'A:HIS:122', 'A:VAL:62', 'A:LEU:109',                           
                             'A:THR:38', 'A:PHE:128', 'A:ALA:26', 'A:PHE:98',                
                             'A:SER:52', 'A:LEU:66', 'A:VAL:93', 'A:LYS:40',                 
                             'A:VAL:73', 'A:ALA:71', 'A:ALA:123', 'A:PRO:95',                
                             'A:PRO:119', 'A:THR:108', 'A:LYS:7', 'A:ASP:94',                
                             'A:ALA:82', 'A:LEU:136', 'A:THR:41', 'A:VAL:1',                 
                             'A:GLU:27', 'A:SER:81', 'A:SER:35', 'A:ASP:85',                 
                             'A:ASP:126', 'A:ARG:92', 'A:LYS:56', 'A:ASP:47',                
                             'A:VAL:70', 'A:ALA:120', 'A:LEU:48', 'A:THR:67',                
                             'A:ARG:31', 'A:HIS:112', 'A:HIS:89',                            
                             'A:VAL:132', 'A:PHE:43', 'A:ASN:97', 'A:ALA:53',                
                             'A:LEU:29', 'A:PRO:114', 'A:MET:32',                            
                             'A:THR:137', 'A:GLY:18', 'A:LYS:99',                            
                             'A:LYS:127', 'A:TYR:140', 'A:ALA:63',                           
                             'A:ASP:74', 'A:ALA:130', 'A:PRO:37', 'A:TYR:24',                
                             'A:PRO:44', 'A:LEU:80', 'A:TRP:14', 'A:ALA:21',                 
                             'A:ALA:65', 'A:GLY:25', 'A:MET:76', 'A:ALA:19',                 
                             'A:GLN:54', 'A:ALA:13', 'A:LYS:139', 'A:GLU:30',                
                             'A:LYS:16', 'A:THR:134', 'A:VAL:17',                            
                             'A:LEU:113', 'A:HIS:45', 'A:ARG:141',                           
                             'A:VAL:10', 'A:GLU:116', 'A:SER:49',                            
                             'A:LEU:100', 'A:VAL:107', 'A:VAL:121',                          
                             'A:CYS:104', 'A:SER:133', 'A:PHE:46',                           
                             'A:ALA:69', 'A:ALA:115', 'A:HIS:50',                            
                             'A:SER:131', 'A:ASN:68', 'A:GLY:22', 'A:ASN:78',                
                             'A:VAL:135', 'A:SER:84', 'A:PHE:117',                           
                             'A:LEU:83', 'A:ASP:64', 'A:GLY:51', 'A:LYS:11',                 
                             'A:LEU:101', 'A:ASP:6', 'A:LEU:129', 'A:LEU:34',                
                             'A:PRO:4', 'A:GLY:15', 'A:VAL:55', 'A:ALA:111',                 
                             'A:HIS:58', 'A:HIS:72', 'A:SER:102', 'A:ASN:9',                 
                             'A:GLU:23', 'A:LEU:125', 'A:LYS:61', 'A:ALA:28',                
                             'A:HIS:103', 'A:SER:124', 'A:SER:3', 'A:PHE:33',                
                             'A:HIS:20', 'A:ALA:12', 'A:PRO:77', 'A:ALA:5',                  
                             'A:LYS:90', 'A:LEU:2', 'A:LEU:105', 'A:ALA:79',                 
                             'A:THR:39', 'A:LEU:106', 'A:LEU:91', 'A:ASP:75',                
                             'A:GLY:57', 'A:ALA:110', 'A:PHE:36', 'A:LEU:86',                
                             'A:TYR:42', 'A:LYS:60', 'A:SER:138', 'A:THR:8'].                

                    DEBUG    Found 97 nodes in the chain subgraph.           ]8;id=306792;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=937503;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#266\266]8;;\

                    DEBUG    Creating subgraph from nodes: ['A:ASP:53',       ]8;id=481381;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=322522;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#57\57]8;;\
                             'A:PHE:56', 'A:ASP:76', 'A:SER:52', 'A:VAL:93',                 
                             'A:SER:61', 'A:SER:11', 'A:ARG:81', 'A:ASN:83',                 
                             'A:CYS:25', 'A:ILE:92', 'A:VAL:82', 'A:GLU:69',                 
                             'A:LEU:87', 'A:THR:4', 'A:GLU:16', 'A:ASN:42',                  
                             'A:GLY:43', 'A:LYS:48', 'A:TYR:67', 'A:ASN:24',                 
                             'A:SER:57', 'A:LEU:23', 'A:HIS:84', 'A:PRO:90',                 
                             'A:PRO:5', 'A:ILE:7', 'A:TYR:26', 'A:ILE:35',                   
                             'A:GLY:18', 'A:LEU:65', 'A:HIS:51', 'A:ILE:46',                 
                             'A:LYS:19', 'A:ASP:59', 'A:PHE:30', 'A:VAL:49',                 
                             'A:ASP:38', 'A:THR:68', 'A:CYS:80', 'A:LYS:58',                 
                             'A:ASN:21', 'A:GLN:2', 'A:ALA:15', 'A:ASN:17',                  
                             'A:ARG:45', 'A:MET:0', 'A:VAL:27', 'A:GLU:77',                  
                             'A:LYS:94', 'A:VAL:37', 'A:VAL:85', 'A:GLN:89',                 
                             'A:THR:71', 'A:THR:73', 'A:LEU:54', 'A:LYS:75',                 
                             'A:PHE:22', 'A:GLU:47', 'A:ARG:3', 'A:PHE:62',                  
                             'A:GLY:29', 'A:LEU:39', 'A:PRO:14', 'A:THR:86',                 
                             'A:TYR:63', 'A:GLU:50', 'A:SER:55', 'A:PRO:72',                 
                             'A:HIS:31', 'A:SER:20', 'A:GLU:74', 'A:LYS:41',                 
                             'A:TYR:10', 'A:GLU:36', 'A:ARG:12', 'A:TRP:95',                 
                             'A:SER:88', 'A:SER:33', 'A:TYR:66', 'A:SER:28',                 
                             'A:LEU:64', 'A:GLU:44', 'A:TYR:78', 'A:HIS:13',                 
                             'A:LYS:91', 'A:ILE:1', 'A:ALA:79', 'A:ASP:96',                  
                             'A:LYS:6', 'A:LEU:40', 'A:TRP:60', 'A:GLN:8',                   
                             'A:VAL:9', 'A:PRO:32', 'A:PHE:70', 'A:ASP:34'].                 

                    DEBUG    Found 148 nodes in the chain subgraph.          ]8;id=748370;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=482323;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#266\266]8;;\

                    DEBUG    Creating subgraph from nodes: ['A:PHE:16',       ]8;id=936948;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py\subgraphs.py]8;;\:]8;id=3908;file:///home/atj39/github/graphein/graphein/protein/subgraphs.py#57\57]8;;\
                             'A:GLU:83', 'A:ASP:22', 'A:GLY:59', 'A:ARG:90',                 
                             'A:LEU:4', 'A:ARG:106', 'A:GLY:132', 'A:ASP:58',                
                             'A:ALA:88', 'A:ARG:37', 'A:MET:71', 'A:LYS:21',                 
                             'A:GLU:114', 'A:ASP:95', 'A:GLY:23', 'A:GLU:82',                
                             'A:ALA:103', 'A:GLU:54', 'A:ALA:147',                           
                             'A:ARG:126', 'A:GLU:45', 'A:THR:28', 'A:ALA:1',                 
                             'A:ALA:46', 'A:ALA:73', 'A:GLU:67', 'A:SER:81',                 
                             'A:GLU:84', 'A:VAL:108', 'A:ASN:42',                            
                             'A:LYS:115', 'A:THR:5', 'A:ILE:63', 'A:ASP:2',                  
                             'A:ASP:80', 'A:ALA:128', 'A:ASP:50', 'A:SER:17',                
                             'A:ASP:129', 'A:VAL:136', 'A:LEU:48',                           
                             'A:PHE:141', 'A:ILE:85', 'A:GLY:33', 'A:MET:51',                
                             'A:GLY:134', 'A:ILE:125', 'A:ASN:97',                           
                             'A:GLU:120', 'A:THR:110', 'A:ASP:93',                           
                             'A:SER:101', 'A:ILE:9', 'A:ASP:20', 'A:ASN:53',                 
                             'A:GLU:127', 'A:VAL:142', 'A:ASP:122',                          
                             'A:GLU:7', 'A:ALA:15', 'A:GLY:25', 'A:VAL:35',                  
                             'A:MET:76', 'A:GLY:61', 'A:LEU:116', 'A:LYS:94',                
                             'A:LEU:18', 'A:THR:34', 'A:GLU:140', 'A:LYS:13',                
                             'A:ARG:74', 'A:PHE:92', 'A:ARG:86', 'A:GLY:98',                 
                             'A:GLY:113', 'A:LEU:69', 'A:THR:29', 'A:MET:36',                
                             'A:LYS:75', 'A:LEU:32', 'A:GLU:123', 'A:GLU:47',                
                             'A:VAL:121', 'A:ASP:24', 'A:THR:44', 'A:THR:62',                
                             'A:ALA:102', 'A:MET:72', 'A:LEU:39', 'A:VAL:91',                
                             'A:PHE:65', 'A:GLU:119', 'A:THR:146',                           
                             'A:GLU:139', 'A:THR:117', 'A:GLN:135',                          
                             'A:GLN:143', 'A:LEU:112', 'A:ASN:137',                          
                             'A:ASP:64', 'A:GLY:40', 'A:ILE:27', 'A:GLY:96',                 
                             'A:MET:109', 'A:GLN:49', 'A:ASP:78', 'A:PHE:89',                
                             'A:ILE:100', 'A:VAL:55', 'A:THR:79', 'A:GLU:6',                 
                             'A:GLU:104', 'A:MET:145', 'A:SER:38',                           
                             'A:GLU:31', 'A:PRO:66', 'A:LYS:148', 'A:GLU:87',                
                             'A:GLU:14', 'A:HIS:107', 'A:MET:144',                           
                             'A:PHE:12', 'A:ASN:60', 'A:MET:124', 'A:ALA:57',                
                             'A:ASP:56', 'A:LYS:77', 'A:GLU:11', 'A:GLN:3',                  
                             'A:ILE:130', 'A:ASP:131', 'A:TYR:138',                          
                             'A:ALA:10', 'A:GLN:41', 'A:LEU:105', 'A:PHE:68',                
                             'A:ASN:111', 'A:THR:70', 'A:PRO:43', 'A:GLN:8',                 
                             'A:LYS:30', 'A:ILE:52', 'A:PHE:19', 'A:ASP:118',                
                             'A:TYR:99', 'A:ASP:133', 'A:THR:26']

In [12]:
for i in ds:
    print(i)

Data(edge_index=[2, 173], node_id=[174], coords=[1], name=[1], dist_mat=[1], num_nodes=174)
Data(edge_index=[2, 140], node_id=[141], coords=[1], name=[1], dist_mat=[1], num_nodes=141)
Data(edge_index=[2, 96], node_id=[97], coords=[1], name=[1], dist_mat=[1], num_nodes=97)
Data(edge_index=[2, 147], node_id=[148], coords=[1], name=[1], dist_mat=[1], num_nodes=148)


In [13]:
# Create a dataloader from dataset and inspect a few batches
dl = DataLoader(ds, batch_size=2, shuffle=True, drop_last=False)
for i in dl:
    print(i)

DataBatch(edge_index=[2, 313], node_id=[2], coords=[2], name=[2], dist_mat=[2], num_nodes=315, batch=[315], ptr=[3])
DataBatch(edge_index=[2, 243], node_id=[2], coords=[2], name=[2], dist_mat=[2], num_nodes=245, batch=[245], ptr=[3])


### Transforms

We can supply various functions to `ProteinGraphDataset` and `InMemoryProteinGraphDataset` to alter the composition of the dataset.

* ``pdb_transform`` (``list(callable)``, optional) - A function that receives a list of paths to the downloaded structures. This provides an entry point to apply pre-processing from bioinformatics tools of your choosing

* ``graph_transformation_funcs``: (``List[Callable]``, optional) List of functions that consume a ``nx.Graph`` and return a ``nx.Graph``. Applied to graphs after construction but before conversion to ``torch_geometric.data.Data``. Defaults to ``None``.

* ``transform`` (``callable``, optional) – A function/transform that takes in a ``torch_geometric.data.Data`` object and returns a transformed version. The data object will be transformed before every access. (default: ``None``)

* ``pre_transform`` (``callable``, optional) – A function/transform that takes in a torch_geometric.data.Data object and returns a transformed version. The data object will be transformed before being saved to disk. (default: ``None``)

* ``pre_filter`` (``callable,`` optional) – A function that takes in a ``torch_geometric.data.Data`` object and returns a boolean value, indicating whether the data object should be included in the final dataset. (default: ``None``)

In [14]:
from typing import List
import networkx as nx
from torch_geometric.data import Data

# Create dummy transforms
def pdb_transform_fn(files: List[str]):
    """Transforms raw pdbs prior to computing graphs."""
    return

def graph_transform_fn(graph: nx.Graph) -> nx.Graph:
    """Transforms graphein nx.Graph prior to conversion to torch_geometric.data.Data."""
    return graph

def transform_fn(data: Data) -> Data:
    """Transforms torch_geometric.data.Data prior to every access."""
    return data

def pre_transform_fn(data: Data) -> Data:
    """Transforms torch_geometric.data.Data prior to saving to disk."""
    return data

def pre_filter_fn(data: Data) -> bool:
    """Takes in a torch_geometric.data.Data and returns True if the data should be included in the dataset."""
    return True

In [15]:
from graphein.ml.datasets.torch_geometric_dataset import InMemoryProteinGraphDataset

g_lab_map = {"3eiy": 1, "4hhb": 2, "Q5VSL9": 3, "1lds": 10, "2ll6": 4}
node_lab_map = {"3eiy": 1, "4hhb": 2, "Q5VSL9": 3, "1lds": 10, "2ll6": 4}
chain_selection_map = {"4hhb": "A"}

ds = InMemoryProteinGraphDataset(
    root = "../graphein/ml/datasets/test",
    name="test",
    pdb_codes=["3eiy", "4hhb", "1lds", "2ll6"],
    uniprot_ids=["Q5VSL9"],
    graph_label_map=g_lab_map,
    node_label_map=node_lab_map,
    chain_selection_map=chain_selection_map,
    pdb_transform=[pdb_transform_fn],
    graph_transformation_funcs=[graph_transform_fn],
    transform=transform_fn,
    pre_transform=pre_transform_fn,
    pre_filter=pre_filter_fn
)

/home/atj39/anaconda3/envs/graphein/lib/python3.8/site-packages/torch_geometric/data/dataset.py:190: UserWarning: The `pre_transform` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-processing technique, make sure to delete '../graphein/ml/datasets/test/processed' first
  warnings.warn(
/home/atj39/anaconda3/envs/graphein/lib/python3.8/site-packages/torch_geometric/data/dataset.py:198: UserWarning: The `pre_filter` argument differs from the one used in the pre-processed version of this dataset. If you want to make use of another pre-fitering technique, make sure to delete '{self.processed_dir}' first
  warnings.warn(
